In [1]:
import pandas as pd
import numpy as np

## Income data

In [5]:
incomeing = pd.read_parquet('./datalake/income_data/2024-04-17_IRSIncomeByZipCode_NoStateTotalsNoSmallZips.parquet')

In [6]:
incomeing.head()

,STATE,ZIPCODE,Number of returns,Adjusted gross income (AGI),Avg AGI,Number of returns with total income,Total income amount,Avg total income,Number of returns with taxable income,Taxable income amount,Avg taxable income
0,AL,35004,4930,255534,51.832454,4930,258024,52.337525,4020,163859,40.760945
1,AL,35005,3300,128387,38.905152,3300,129390,39.209091,2440,70760,29.000000
2,AL,35006,1230,58302,47.400000,1230,58585,47.630081,940,36341,38.660638
3,AL,35007,11990,643708,53.687073,11990,651350,54.324437,9280,414878,44.706681
4,AL,35010,8320,378497,45.492428,8320,382106,45.926202,5610,226514,40.376827


In [4]:
incomeing["STATE"].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [62]:
abreviaciones_estados = [
    "AL", "AK", "AZ", "AR", "CA", 
    "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", 
    "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", 
    "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", 
    "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", 
    "VA", "WA", "WV", "WI", "WY"
]


In [65]:
def diferencia_listas(lista1, lista2):
    return [elemento for elemento in lista1 if elemento not in lista2]

# Ejemplo de uso
lista_a = [1, 2, 3, 4, 5]
lista_b = [4, 5, 6, 7, 8]

resultado = diferencia_listas(incomeing["STATE"].unique(),abreviaciones_estados)
print("Diferencia de lista_a y lista_b:", resultado)


Diferencia de lista_a y lista_b: ['DC']


In [64]:
len(incomeing["STATE"].unique())

51

## Sales data

In [6]:
sales = pd.read_csv('./datalake/sales_data/2024-04-11_SuperstoreSalesTraining.csv', encoding='latin1')

sales.head()

,Row,Order Priority,Order Date,Order,Discount,Unit Price,Order Quantity,Sales,Profit,Shipping Cost,...,Customer_ID,Customer Name,Region,State,Country / Region,City,Postal Code,Ship Date,Ship Mode,SubRegion
0,1,High,01/01/2010,28774,10%,6.0,32,172.80,106.36,5.0,...,1656,Joy Corbett,AsiaPac,Central,Fiji,Suva,NaN,02/01/2010,Regular Air,NaN
1,2,Not Specified,01/01/2010,88028,8%,96.0,2,176.64,45.64,35.0,...,2211,Anita Hahn,North America,Maryland,United States of America,Bowie,20715.0,03/01/2010,Express Air,East ...
2,3,Critical,02/01/2010,9285,6%,41.0,3,115.62,33.90,3.0,...,949,Ernest Oh,North America,California,United States of America,Los Angeles,90049.0,04/01/2010,Regular Air,West ...
3,4,Low,02/01/2010,37537,0%,292.0,4,1168.00,605.08,49.0,...,68,Scott Bunn,North America,New York,United States of America,New York City,10177.0,02/01/2010,Delivery Truck,East ...
4,5,Low,02/01/2010,37537,7%,101.0,43,4038.99,2647.66,45.0,...,68,Scott Bunn,North America,New York,United States of America,New York City,10177.0,04/01/2010,Delivery Truck,East ...


### Dades amb missings:
- Postal Code
- SubRegion

Com tractar-les?? Si té subregion, agafar el codi postar de la subregió, si no en té, agafar un codi postal de la regió. 
En teoria, si ja es té el codi postal no caldria la variable subregion.

In [7]:
sales['Postal Code']

0            NaN
1        20715.0
2        90049.0
3        10177.0
4        10177.0
          ...   
16793    94533.0
16794        NaN
16795    60174.0
16796    14120.0
16797    14120.0
Name: Postal Code, Length: 16798, dtype: float64

In [ ]:
#Variable Item, 1263 valors diferents
#Customer Name, 3403 valors diferents

In [ ]:
#Variables -> Country / Region, Region, State -> filtrar per a que només aparegui estats units/nord amèrica

## Shops data

In [42]:
import json

with open('./datalake/shops_data/2024-03-22_shops_data.json') as f:
    data = json.load(f)



ValueError: All arrays must be of the same length

In [58]:
data['fields']

[{'name': 'objectid',
  'type': 'esriFieldTypeOID',
  'alias': 'objectid',
  'sqlType': 'sqlTypeOther',
  'domain': None,
  'defaultValue': None},
 {'name': 'osm_id2',
  'type': 'esriFieldTypeString',
  'alias': 'osm_id2',
  'sqlType': 'sqlTypeOther',
  'length': 255,
  'domain': None,
  'defaultValue': None},
 {'name': 'abandoned',
  'type': 'esriFieldTypeString',
  'alias': 'abandoned',
  'sqlType': 'sqlTypeOther',
  'length': 255,
  'domain': None,
  'defaultValue': None},
 {'name': 'addr_housename',
  'type': 'esriFieldTypeString',
  'alias': 'addr_housename',
  'sqlType': 'sqlTypeOther',
  'length': 255,
  'domain': None,
  'defaultValue': None},
 {'name': 'addr_housenumber',
  'type': 'esriFieldTypeString',
  'alias': 'addr_housenumber',
  'sqlType': 'sqlTypeOther',
  'length': 255,
  'domain': None,
  'defaultValue': None},
 {'name': 'addr_street',
  'type': 'esriFieldTypeString',
  'alias': 'addr_street',
  'sqlType': 'sqlTypeOther',
  'length': 255,
  'domain': None,
  'defaul

In [76]:
attributes = pd.DataFrame(data['features']['attributes'])
geometry = pd.DataFrame(data['features']['geometry'])

TypeError: list indices must be integers or slices, not str

In [81]:
a = data['features']


In [67]:
df

,attributes,geometry
0,"{'objectid': 85, 'osm_id2': '6448753092', 'aba...","{'x': -17774397.4774402, 'y': 2506253.3864803435}"
1,"{'objectid': 95, 'osm_id2': '6241439285', 'aba...","{'x': -17774176.6752, 'y': 2506160.271499999}"
2,"{'objectid': 98, 'osm_id2': '4643342514', 'aba...","{'x': -17774121.2047, 'y': 2506174.398400001}"
3,"{'objectid': 105, 'osm_id2': '5857253155', 'ab...","{'x': -17774882.9195, 'y': 2506481.507199999}"
4,"{'objectid': 107, 'osm_id2': '5541678538', 'ab...","{'x': -17774423.815631725, 'y': 2506268.521614..."
...,...,...
1995,"{'objectid': 37727, 'osm_id2': '3218142379', '...","{'x': -11719036.2628, 'y': 2355205.7261000015}"
1996,"{'objectid': 37728, 'osm_id2': '3218142341', '...","{'x': -11719019.5203, 'y': 2355267.0445999987}"
1997,"{'objectid': 37730, 'osm_id2': '3218142323', '...","{'x': -11718939.2478, 'y': 2355033.6711999997}"
1998,"{'objectid': 37731, 'osm_id2': '3218142215', '...","{'x': -11718928.694699999, 'y': 2355029.863400..."


In [71]:
import json
import csv
campos_deseados = [
    'addr_city', 'addr_conscriptionnumber', 'addr_country', 'addr_housenumber',
    'addr_postcode', 'addr_province', 'addr_state', 'addr_street', 'addr_suburb',
    'addr_unit', 'amenity', 'brand', 'building', 'description', 'level', 'name',
    'name_en', 'name_es', 'name_fr', 'opening_hours', 'operator', 'phone', 'shop',
    'source_transform', 'source_date', 'website', 'wheelchair', 'osm_id2'
]


# Cargar el archivo JSON
with open('./datalake/shops_data/2024-03-22_shops_data.json', 'r') as file:
    data = json.load(file)

# Abrir un archivo CSV para escribir los datos
with open('./datalake/shops_data/shops_data.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csvfile, fieldnames=campos_deseados)
    writer.writeheader()
    
    # Itera sobre cada feature en los datos JSON
    for feature in data.get('features', []):
        attributes = feature.get('attributes', {})
        
        # Filtra y escribe solo los campos deseados
        row = {field: attributes.get(field) for field in campos_deseados}
        writer.writerow(row)

print("La conversión a CSV ha sido completada.")

NameError: name 'csvfile' is not defined

In [72]:
import json
import csv

# Define las variables que te interesan
campos_deseados = [
    'addr_city', 'addr_conscriptionnumber', 'addr_country', 'addr_housenumber',
    'addr_postcode', 'addr_province', 'addr_state', 'addr_street', 'addr_suburb',
    'addr_unit', 'amenity', 'brand', 'building', 'description', 'level', 'name',
    'name_en', 'name_es', 'name_fr', 'opening_hours', 'operator', 'phone', 'shop',
    'source_transform', 'source_date', 'website', 'wheelchair', 'osm_id2'
]

# Abre y carga el archivo JSON
with open('./datalake/shops_data/2024-03-22_shops_data.json', 'r') as file:
    data = json.load(file)

# Crea y abre un archivo CSV para escribir los datos
with open('./datalake/shops_data/shops_data.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=campos_deseados)
    writer.writeheader()
    
    # Itera sobre cada feature en los datos JSON
    for feature in data.get('features', []):
        attributes = feature.get('attributes', {})
        
        # Filtra y escribe solo los campos deseados
        row = {field: attributes.get(field) for field in campos_deseados}
        writer.writerow(row)

print("La conversión a CSV ha sido completada.")


UnicodeEncodeError: 'charmap' codec can't encode character '\u0101' in position 16: character maps to <undefined>

In [75]:
import json
import csv

# Cargar el archivo JSON
with open('./datalake/shops_data/2024-03-22_shops_data.json', 'r') as file:
    datos = json.load(file)

# Crear y escribir en un archivo CSV, especificando el encoding UTF-8
with open('datos_convertidos.csv', 'w', newline='', encoding='utf-8') as archivo_csv:  # Especifica el encoding aquí.
    campos = [campo['name'] for campo in datos['fields']] + ['x', 'y']
    escritor_csv = csv.DictWriter(archivo_csv, fieldnames=campos)
    
    escritor_csv.writeheader()
    
    for feature in datos['features']:
        fila = feature['attributes']
        fila.update({
            'x': feature['geometry']['x'],
            'y': feature['geometry']['y']
        })
        escritor_csv.writerow(fila)
